# Setting up Cloud Observability with Terraform - Provisioning Health Rules with Terraform 

In the previous sections, we reviewed using API's to set up cloud connections, health rules, anomaly detection to AWS cloud resources. In this section, we shall use Terraform to do the same.
    
To progress with the lab, click **inside** the boxes below that contain the snippets of Python code. Note that the **Run** button will appear to the left of the code box. Click on the **arrow** button to execute the code snippet. After you click **Run**, executions in progress will be denoted by "ln[*]"

> **Note:** Do not skip code blocks. Execute each before you progress to the next.

> **Note:** Wait for the entire execution to complete in a code block before you progress to the next.

## Pre-execution steps

Recall the globals set in in the previous notebook. Click **Run** to set the environment.

In [ ]:
import os
%store -r aws_key
%store -r aws_secret
%store -r aws_conn_name
%store -r cl_id
%store -r cl_secret

os.environ['AWS_KEY'] = aws_key
os.environ['AWS_SECRET'] = aws_secret
os.environ['AWS_CONN_NAME'] = aws_conn_name
os.environ['CL_ID'] = cl_id
os.environ['CL_SECRET'] = cl_secret
print("Globals set:" + "\n")
print("Connection Name:" + aws_conn_name + "\n")
print("AWS Secret:" + aws_secret + "\n")
print("AWS Key:" + aws_key + "\n")


## Check Terraform and initialize AppDynamics Provider

Run this command to make sure you can access Terraform that has been set up for this environment. Also, note that we are using a local copy of the AppDynamics Terraform provider and we shall copy it to your local instance.

In [ ]:
!terraform version


In [ ]:
!mkdir -p ~/.terraform.d/plugins/terraform.local/local/appdynamics/1.0.0/darwin_amd64/terraform-provider-appdynamics_v1.0.0


In [ ]:
!cp ./plugin/appdynamicscloud ~/.terraform.d/plugins/terraform.local/local/appdynamics/1.0.0/darwin_amd64/terraform-provider-appdynamics_v1.0.0


In [ ]:
!rm main.tf data.tf dataregions.tf dataservices.tf datas.tf allconns.tf

## Create Health Rule Terraform Configuration

Lets first write the paylod for the Health Rule and then apply the terraform plan.

In [ ]:
%%writefile createHR.json

{
    "name": "Create HR Using terraform Script",
    "description": "Health Rule : Create HR Using terraform Script",
    "enabled": true,
    "scheduleName": "Always",
    "waitTimeAfterViolation": "5m",
    "evaluationObjects": "k8s:pod()",
    "criticalCriteria": {
        "criteriaExpression": "A",
        "conditions": [
            {
                "name": "Condition 1",
                "label": "A",
                "conditionExpression": "metric:k8s.memory.usage.cmin(2m)>95",
                "evaluateToTrueOnNoData": false,
                "source": "infra-agent"
            }
        ]
    },
    "warningCriteria": {
        "criteriaExpression": "A",
        "conditions": [
            {
                "name": "Condition 1",
                "label": "A",
                "conditionExpression": "metric:k8s.memory.usage.cmin(2m)>75",
                "evaluateToTrueOnNoData": false,
                "source": "infra-agent"
            }
        ]
    },
    "associateHealthTo": [
        {
            "objectExpr": "this"
        }
    ]
}


In [ ]:
%%writefile createHR.tf

terraform {
  required_providers {
    http = {
      source = "hashicorp/http"
      version = "3.1.0"
    }
    local = {
      source = "hashicorp/local"
      version = "2.2.3"
    }
  }
}

provider "http" {
  # Configuration options
}


data "http" "get_auth_token" {
  provider = http
  url = "${var.appdURL}/auth/${var.tenant}/default/oauth2/token"

  request_headers = {
    Content-Type  =  "application/x-www-form-urlencoded"
    Authorization = "Basic ${base64encode("${var.clientId}:${var.clientSecret}")}"
  }
  request_body = "grant_type=client_credentials"
  method = "POST"
  lifecycle {
    postcondition {
      condition     = contains([200], self.status_code)
      error_message = "Status code invalid, Auth Token not generated successfully. ${self.response_body}"
    }
  }
}


locals {
  auth_response = jsondecode(data.http.get_auth_token.response_body)
  json_data = file("${path.module}/createHR.json")
}



data "http" "create_healthrule"{
provider = http
  url = "${var.appdURL}/alerting/v1beta/healthRules"
  request_headers = {
    Content-Type   = "application/json"
    Authorization  ="Bearer ${local.auth_response.access_token}"
    Appd-Tenant-Id = "${var.tenant}"
    Accept         = "application/json"
  }
  method = "POST"
  request_body = "${local.json_data}"
}

output "get_response" {
  value = "Health rule create response is ${data.http.create_healthrule.response_body} with response code ${data.http.create_healthrule.status_code}"
}

resource "local_file" "json-data" {
    content  = data.http.create_healthrule.response_body
    filename = "CreateHRResponse.json"
}


In [ ]:
%%writefile variables.tf

variable "appdURL" {
  description = "Endpoint of the Cloud Native Application Observability applications"
  type        = string
  default     = "<Your FSO Tenant URL>"
}

variable "tenant" {
  description = "Tenant ID"
  type        = string
  default     = "<Your FSO Tenant ID as per your Service Principals>"
}

variable "clientId" {
  description = "Generated Client ID"
  type        = string
  default     = "<Your Client ID as per your Service Principals>"
}

variable "clientSecret" {
  description = "Generated Client Secret"
  type        = string
  default     = "<Your Client Secret Key as per your Service Principals>"
}

## Initialize Terraform

In [ ]:
!terraform init

## Checkout Terraform Plan

In [ ]:
!terraform plan -var="access_key_id=$aws_key" -var="secret_access_key=$aws_secret" -var="client_id=$cl_id" -var="client_secret=$cl_secret" -var="conn_name=$aws_conn_name" -var="descr=XXX"


## Apply Terraform Plan to Create Health Rule

In [ ]:
!terraform apply -auto-approve -var="conn_name=$aws_conn_name" -var="descr=XXX" -var="access_key_id=$aws_key" -var="secret_access_key=$aws_secret" -var="client_id=$cl_id" -var="client_secret=$cl_secret"


In [ ]:
connid="$(terraform output -raw conn_id)"
!terraform apply -auto-approve  -var="tenant=76d947c5-91ee-4a95-8833-7bf9040a3d88" -var="appdURL=https://cisco-devnet.observe.appdynamics.com" -var="descr=Test connection for AWS" -var="conn_name=$aws_conn_name" -var="access_key_id=$aws_key" -var="secret_access_key=$aws_secret" -var="client_id=$cl_id" -var="client_secret=$cl_secret"


## Delete Health Rule


In [ ]:
!terraform destroy -auto-approve -var="conn_name=$aws_conn_name" -var="descr=XXX" -var="access_key_id=$aws_key" -var="secret_access_key=$aws_secret" -var="client_id=$cl_id" -var="client_secret=$cl_secret"
